# Shor's algorithm for computing discrete logarithms
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's algorithm for computing general discrete logarithms [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084), and with the classical post-processing in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

To start off, let us define the P-384 curve $E$ and associated generator $g$ of order $r$ as specified in [NIST SP 800-186](https://doi.org/10.6028/NIST.SP.800-186).

In [ ]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [1]:
from quaspy.math.groups import ShortWeierstrassCurveOverPrimeField;
from quaspy.math.groups import PointOnShortWeierstrassCurveOverPrimeField;

# Define the elliptic curve.
p = 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319;

a = -3;
b = 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575;

E = ShortWeierstrassCurveOverPrimeField(a, b, p);

# Define the generator.
g_x = 26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087;
g_y = 8325710961489029985546751289520108179287853048861315594709205902480503199884419224438643760392947333078086511627871;

g = PointOnShortWeierstrassCurveOverPrimeField(g_x, g_y, E);

# Define the order of the generator.
r = 39402006196394479212279040100143613805079739270465446667946905279627659399113263569398956308152294913554433653942643;

Let us continue to sample the exponent $d$ from $[0, r) \cap \mathbb Z$.

To this end, we use the [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [2]:
from quaspy.math.random import sample_integer;

# Sample d.
m = 384;
d = sample_integer(r);

print("Sampled d =", d);

# Compute x.
x = g ** d;

print("Computed x =", x);

Sampled d = 26046655994679286574374766241369555976235272518257515665412510485456329664735736163500845066158257105208337102748983
Computed x = (38094112960201363581443997052505814464706699016090154015286646959348949038135785282498024756212537717005953159209414, 30297166203074667741662041277094715915784574365901105383678323334575282408637202058228593815868634001207317916673801) on y^2 = x^3 + 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112316 x + 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575 (mod 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319)


## 1. Simulating the quantum part of Shor's algorithm
We may now proceed to use the [<code>sample_j_k_given_d_r_heuristic(d, r, m, sigma, l, ..)</code>](../docs/logarithmfinding/sampling/sample_j_k_given_d_r_heuristic.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to heuristically simulate the quantum part of Shor's algorithm for computing discrete logarithms when modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

Below, we use this function to simulate running the algorithm for $d$ and $r$ with control registers of length $m + \sigma$ bits and $\ell$ bits respectively.

This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the quantum algorithm:

In [3]:
from quaspy.logarithmfinding.sampling import sample_j_k_given_d_r_heuristic;

sigma = 0;
l = m + sigma;

[j, k] = sample_j_k_given_d_r_heuristic(d, r, m, sigma, l);

print("Sampled j =", j);
print("Sampled k =", k);

Sampled j = 15478387178455032864378871666556073024000828228842641764920079573660068220935466213618668977828979067324798308331576
Sampled k = 81447030165655034437254060944243733002916920223995551773468391363517124218955370555990875744135416284979306219370


### 1.1 Solving the frequency pair $(j, k)$ for $d$
We may now execute the classical post-processing algorithm from [[E19p]](https://doi.org/10.48550/arXiv.1905.09084) to recover $d$ from $(j, k)$.

To this end, we use the [<code>solve_j_k_for_d_given_r(j, k, m, sigma, l, g, x, r, ..)</code>](../docs/logarithmfinding/general/postprocessing/solve_j_k_for_d_given_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

In [4]:
from quaspy.logarithmfinding.general.postprocessing import solve_j_k_for_d_given_r;

recovered_d = solve_j_k_for_d_given_r(j, k, m, sigma, l, g, x, r);

if (recovered_d == d):
  print("Recovered d =", d);

  print("\n[ OK ] Successfully recovered d.");
else:
  print("[FAIL] Failed to recover d.");

Recovered d = 26046655994679286574374766241369555976235272518257515665412510485456329664735736163500845066158257105208337102748983

[ OK ] Successfully recovered d.
